# Reading images

Astropy provides a few ways to read in FITS images, some in the core package and
others in affiliated packages.

Before exploring those, we'll create a set of (fake) images to work with.

In [1]:
from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits

## Working with directories

[*Click here to comment on this section on GitHub (opens in new tab).*](https://github.com/mwcraig/ccd-reduction-and-photometry-guide/pull/206/files#diff-487933c57483df376c3cb4974dd2f82cR30){:target="_blank"}

The cell below contains the path to the images. In this notebook we'll use it
both to store the fake images we generate and to read images. In normal use, you
wouldn't start by writing images there, however.

If the images are in the same directory as the notebook you can omit this, or
set it to an empty string `''`. Having images in the same directory as the
notebook is less complicated, but it's not at all uncommon to need to work with
images in a different directory.

Later, we'll look at how to generate the full path to an image (directory plus
file name) in a way that will work on any platform. One of the approaches to
loading images (using `ccdproc.ImageFileCollection`) lets you mostly forget
about this.

In [2]:
data_directory = 'path/to/my/images'

## Generate some fake images

[*Click here to comment on this section on GitHub (opens in new tab).*](https://github.com/mwcraig/ccd-reduction-and-photometry-guide/pull/206/files#diff-487933c57483df376c3cb4974dd2f82cR60){:target="_blank"}

The cells below generate some fake images to use later in the notebook.

In [3]:
from pathlib import Path
from itertools import cycle

import numpy as np

image_path = Path(data_directory)

image_path.mkdir(parents=True, exist_ok=True)

images_to_generate = {
    'BIAS': 5,
    'DARK': 10,
    'FLAT': 3,
    'LIGHT': 10
}

exposure_times = {
    'BIAS': [0.0],
    'DARK': [5.0, 30.0],
    'FLAT': [5.0, 6.1, 7.3],
    'LIGHT': [30.0],
}

filters = {
    'FLAT': 'V',
    'LIGHT': 'V'
}

objects = {
    'LIGHT': ['m82', 'xx cyg']
}

image_size = [300, 200]

image_number = 0
for image_type, num in images_to_generate.items():
    exposures = cycle(exposure_times[image_type])
    try:
        filts = cycle(filters[image_type])
    except KeyError:
        filts = []
    
    try:
        objs = cycle(objects[image_type])
    except KeyError:
        objs = []
    for _ in range(num):
        img = CCDData(data=np.random.randn(*image_size), unit='adu')
        img.meta['IMAGETYP'] = image_type
        img.meta['EXPOSURE'] = next(exposures)
        if filts:
            img.meta['FILTER'] = next(filts)
        if objs:
            img.meta['OBJECT'] = next(objs)
        image_name = str(image_path / f'img-{image_number:04d}.fits')
        img.write(image_name)
        print(image_name)
        image_number += 1

path/to/my/images/img-0000.fits
path/to/my/images/img-0001.fits
path/to/my/images/img-0002.fits
path/to/my/images/img-0003.fits
path/to/my/images/img-0004.fits
path/to/my/images/img-0005.fits
path/to/my/images/img-0006.fits
path/to/my/images/img-0007.fits
path/to/my/images/img-0008.fits
path/to/my/images/img-0009.fits
path/to/my/images/img-0010.fits
path/to/my/images/img-0011.fits
path/to/my/images/img-0012.fits
path/to/my/images/img-0013.fits
path/to/my/images/img-0014.fits
path/to/my/images/img-0015.fits
path/to/my/images/img-0016.fits
path/to/my/images/img-0017.fits
path/to/my/images/img-0018.fits
path/to/my/images/img-0019.fits
path/to/my/images/img-0020.fits
path/to/my/images/img-0021.fits
path/to/my/images/img-0022.fits
path/to/my/images/img-0023.fits
path/to/my/images/img-0024.fits
path/to/my/images/img-0025.fits
path/to/my/images/img-0026.fits
path/to/my/images/img-0027.fits


## Option 1: Reading a single image with `astropy.io.fits`

[*Click here to comment on this section on GitHub (opens in new tab).*](https://github.com/mwcraig/ccd-reduction-and-photometry-guide/pull/206/files#diff-487933c57483df376c3cb4974dd2f82cR135){:target="_blank"}

This option gives you the most flexibility but is the least adapted to CCD
images specifically. What you read in is a list of FITS extensions; you must
first select the one you want then access the data or header as desired.

We'll open up the first of the fake images, `img-0001.fits`. To combine that
with the directory name we'll use Python 3's `pathlib`, which ensures that the
path combination will work on Windows too.

In [4]:
image_name = 'img-0001.fits'

image_path = Path(data_directory) / image_name

hdu_list = fits.open(image_path)
hdu_list.info()

Filename: path/to/my/images/img-0001.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       8   (200, 300)   float64   


The `hdu_list` is a list of FITS Header-Data Units. In this case there is just
one, containing both the image header and data, which can be accessed as shown
below.

In [5]:
hdu = hdu_list[0]
hdu.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  200                                                  
NAXIS2  =                  300                                                  
IMAGETYP= 'BIAS    '                                                            
EXPOSURE=                  0.0                                                  
BUNIT   = 'adu     '                                                            

In [6]:
hdu.data

array([[ 1.88549491,  0.00598049, -0.82128781, ..., -0.28810373,
        -1.9759894 ,  0.02898905],
       [-0.5175455 , -0.69480248,  0.24895757, ...,  1.07729033,
         0.25320893,  0.00374187],
       [ 0.07851889, -0.81167019,  1.52637413, ..., -0.61393081,
        -1.37688791, -0.90493696],
       ...,
       [ 0.81708177,  0.15023039,  0.11680317, ..., -0.97163781,
         0.42258174,  0.19080197],
       [ 1.41892105,  0.79495699, -0.21972416, ...,  0.17599439,
        -0.17416792,  0.37429249],
       [-0.18603169,  0.42726363, -0.40210443, ...,  0.05583236,
         0.64394215,  0.37381265]])

The [documentation for io.fits](https://astropy.readthedocs.io/en/stable/io/fits/index.html) describes more of its capabilities.

## Option 2:  Use `CCDData` to read in a single image

[*Click here to comment on this section on GitHub (opens in new tab).*](https://github.com/mwcraig/ccd-reduction-and-photometry-guide/pull/206/files#diff-487933c57483df376c3cb4974dd2f82cR199){:target="_blank"}

Astropy contains a `CCDData` object for representing a single image. It's not as
flexible as using `astrop.io.fits` directly (for example, it assumes there is
only one FITS extension and that it contains image data) but it sets up several
properties that make the data easier to work with.

We'll read in the same single image we did in the example above,
`img-0001.fits`.

In [7]:
ccd = CCDData.read(image_path)

The data and header are accessed similarly to how you access it in an HDU
returned by `astropy.io.fits`:

In [8]:
ccd.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  200                                                  
NAXIS2  =                  300                                                  
IMAGETYP= 'BIAS    '                                                            
EXPOSURE=                  0.0                                                  
BUNIT   = 'adu     '                                                            

In [9]:
ccd.data

array([[ 1.88549491,  0.00598049, -0.82128781, ..., -0.28810373,
        -1.9759894 ,  0.02898905],
       [-0.5175455 , -0.69480248,  0.24895757, ...,  1.07729033,
         0.25320893,  0.00374187],
       [ 0.07851889, -0.81167019,  1.52637413, ..., -0.61393081,
        -1.37688791, -0.90493696],
       ...,
       [ 0.81708177,  0.15023039,  0.11680317, ..., -0.97163781,
         0.42258174,  0.19080197],
       [ 1.41892105,  0.79495699, -0.21972416, ...,  0.17599439,
        -0.17416792,  0.37429249],
       [-0.18603169,  0.42726363, -0.40210443, ...,  0.05583236,
         0.64394215,  0.37381265]])

There are a [number of features of `CCDData`](https://astropy.readthedocs.io/en/stable/nddata/ccddata.html) that make it convenient for working
with WCS, slicing, and more. Some of those features will be discussed in more
detail in the notebooks that follow.

## Option 3: Working with a directory of images using `ImageFileCollection`

[*Click here to comment on this section on GitHub (opens in new tab).*](https://github.com/mwcraig/ccd-reduction-and-photometry-guide/pull/206/files#diff-487933c57483df376c3cb4974dd2f82cR258){:target="_blank"}

The affiliated package [ccdproc](https://ccdproc.readthedocs.io/) provides an easier way
to work with collections of images in a directory: an `ImageFileCollection`. The
`ImageFileCollection` is initialized with the name of the directory containing
the images.

In [10]:
from ccdproc import ImageFileCollection
im_collection = ImageFileCollection(data_directory)

Note that we didn't need to worry about using `pathlib` to combine the directory
and file name, instead we give the collection the name of the directory.

### Summary of directory contents

[*Click here to comment on this section on GitHub (opens in new tab).*](https://github.com/mwcraig/ccd-reduction-and-photometry-guide/pull/206/files#diff-487933c57483df376c3cb4974dd2f82cR288){:target="_blank"}

The `summary` property provides an overview of the files in the directory: it's
an astropy `Table`, so you can access columns in the usual way.

In [11]:
im_collection.summary

file,simple,bitpix,naxis,naxis1,naxis2,imagetyp,exposure,bunit,filter,object
str13,bool,int64,int64,int64,int64,str5,float64,str3,object,object
img-0000.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0001.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0002.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0003.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0004.fits,True,-64,2,200,300,BIAS,0.0,adu,--,--
img-0005.fits,True,-64,2,200,300,DARK,5.0,adu,--,--
img-0006.fits,True,-64,2,200,300,DARK,30.0,adu,--,--
img-0007.fits,True,-64,2,200,300,DARK,5.0,adu,--,--
img-0008.fits,True,-64,2,200,300,DARK,30.0,adu,--,--


### Filtering and iterating over images

[*Click here to comment on this section on GitHub (opens in new tab).*](https://github.com/mwcraig/ccd-reduction-and-photometry-guide/pull/206/files#diff-487933c57483df376c3cb4974dd2f82cR307){:target="_blank"}

The great thing about `ImageFileCollection` is that it provides convenient ways
to filter or loop over files via FITS header keyword values.

For example, looping over just the flat files is one line of code:

In [12]:
for a_flat in im_collection.hdus(imagetyp='FLAT'):
    print(a_flat.header['EXPOSURE'])

5.0
6.1
7.3


Instead of iterating over HDUs, as in the example above, you can iterate over
just the headers (with `.headers`) or just the data (with `.data`). You can use
any FITS keyword from the header as a keyword for selecting the images you want.
In addition, you can return the file name while also iterating.

In [13]:
for a_flat, fname in im_collection.hdus(imagetyp='LIGHT', object='m82', return_fname=True):
    print(f'In file {fname} the exposure is:', a_flat.header['EXPOSURE'], 'with standard deviation ', a_flat.data.std())

In file img-0018.fits the exposure is: 30.0 with standard deviation  0.9982809897572352
In file img-0020.fits the exposure is: 30.0 with standard deviation  1.002667220369337
In file img-0022.fits the exposure is: 30.0 with standard deviation  1.0002050007963355
In file img-0024.fits the exposure is: 30.0 with standard deviation  0.9981232307305272
In file img-0026.fits the exposure is: 30.0 with standard deviation  1.0010826099252885


The [documentation for `ImageFileCollection`](https://ccdproc.readthedocs.io/en/latest/ccdproc/image_management.html) describes more of its capabilities.
`ImageFileCollection` can automatically save a copy of each image as you iterate
over them, for example.

In [14]:
for a_flat, fname in im_collection.ccds(bunit='ADU', return_fname=True):
    print(a_flat.unit)

adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu
adu


In [15]:
a_flat.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  200                                                  
NAXIS2  =                  300                                                  
IMAGETYP= 'LIGHT   '                                                            
EXPOSURE=                 30.0                                                  
FILTER  = 'V       '                                                            
OBJECT  = 'xx cyg  '                                                            
BUNIT   = 'adu     '                                                            